In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_path = "Resources/schools_complete.csv"
student_data_path = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_path)
student_data = pd.read_csv(student_data_path)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on="school_name")
school_data_complete

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


In [2]:
school_names = school_data_complete["school_name"].unique()
total_schools = len(school_names)
total_students = len(school_data_complete["Student ID"])
total_budget = school_data["budget"].sum()
avg_math = school_data_complete["math_score"].mean()
avg_reading = school_data_complete["reading_score"].mean()
math_passes = [score for score in school_data_complete["math_score"] if score >= 70]
math_pass_pct = len(math_passes)/total_students
reading_passes = [score for score in school_data_complete["reading_score"] if score >= 70]
reading_pass_pct = len(reading_passes)/total_students
math_passes2 = school_data_complete.loc[school_data_complete["math_score"] >= 70]
pass_both = math_passes2.loc[math_passes2["reading_score"] >= 70]
total_passes = len(pass_both["Student ID"])
passing_pct = total_passes/total_students
district_dict ={"Total Schools":total_schools, "Total Students":total_students, "Total Budget":total_budget,
               "Average Math Score":avg_math, "Average Reading Score":avg_reading,
               "% Passing Math":100*math_pass_pct, "% Passing Reading":100*reading_pass_pct,
               "% Overall Passing":100*passing_pct}
district_summary_df = pd.DataFrame([district_dict])
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


In [46]:
school_summary = school_data.assign(Per_student_budget = school_data['budget']/school_data['size'])
school_summary = school_summary.rename(columns={"school_name":"School Name","type":"School Type","size":"Total Students",
                                                "budget":"Total School Budget","Per_student_budget":"Per Student Budget"})
school_summary = school_summary.drop(columns=['School ID'])

avg_math_list = list()
avg_reading_list = list()
pct_math_list = list()
pct_reading_list = list()
pct_overall_list = list()
for school in school_summary['School Name']:
    temp = student_data.loc[student_data['school_name']==school]
    temp_student_count = len(temp['student_name'])
    temp_math = temp['math_score'].mean()
    temp_reading = temp['reading_score'].mean()
    avg_math_list.append(temp_math)
    avg_reading_list.append(temp_reading)
    
    temp_math_passes = 0
    temp_reading_passes = 0
    for score in temp['math_score']:
        if score >= 70:
            temp_math_passes += 1
    for score in temp['reading_score']:
        if score >= 70:
            temp_reading_passes += 1
    pct_math_list.append(100*temp_math_passes/temp_student_count)
    pct_reading_list.append(100*temp_reading_passes/temp_student_count)
    
    temp2 = temp.loc[temp['math_score'] >= 70]
    temp3 = temp2.loc[temp2['reading_score'] >= 70]
    temp_overall_passes = len(temp3['student_name'])
    pct_overall_list.append(100*temp_overall_passes/temp_student_count)
        
school_summary['Average Math Score'] = avg_math_list
school_summary['Average Reading Score'] = avg_reading_list
school_summary['% Passing Math'] = pct_math_list
school_summary['% Passing Reading'] = pct_reading_list
school_summary['% Overall Passing'] = pct_overall_list

school_summary_formatted = school_summary
school_summary_formatted["Total School Budget"] = school_summary["Total School Budget"].map("${:,.2f}".format)
school_summary_formatted["Per Student Budget"] = school_summary["Per Student Budget"].map("${:,.2f}".format)

school_summary_formatted = school_summary_formatted.set_index('School Name')
school_summary_formatted = school_summary_formatted.sort_index()
school_summary_formatted

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [47]:
schools_by_overall = school_summary_formatted.sort_values(by='% Overall Passing',ascending=False)
top_schools = schools_by_overall.head()
top_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [48]:
bottom_schools = schools_by_overall.tail()
bottom_schools = bottom_schools.sort_values(by='% Overall Passing')
bottom_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [6]:
students_9th = student_data.loc[student_data['grade'] == '9th']
students_9th_grouped = students_9th.groupby('school_name')
students_9th_avg = students_9th_grouped.mean()

students_10th = student_data.loc[student_data['grade'] == '10th']
students_10th_grouped = students_10th.groupby('school_name')
students_10th_avg = students_10th_grouped.mean()

students_11th = student_data.loc[student_data['grade'] == '11th']
students_11th_grouped = students_11th.groupby('school_name')
students_11th_avg = students_11th_grouped.mean()

students_12th = student_data.loc[student_data['grade'] == '12th']
students_12th_grouped = students_12th.groupby('school_name')
students_12th_avg = students_12th_grouped.mean()

math_9th_avg = students_9th_avg['math_score']
math_10th_avg = students_10th_avg['math_score']
math_11th_avg = students_11th_avg['math_score']
math_12th_avg = students_12th_avg['math_score']

math_by_grade = pd.DataFrame({'9th':math_9th_avg,'10th':math_10th_avg,'11th':math_11th_avg,'12th':math_12th_avg})
math_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [7]:
reading_9th_avg = students_9th_avg['reading_score']
reading_10th_avg = students_10th_avg['reading_score']
reading_11th_avg = students_11th_avg['reading_score']
reading_12th_avg = students_12th_avg['reading_score']

reading_by_grade = pd.DataFrame({'9th':reading_9th_avg,'10th':reading_10th_avg,
                                 '11th':reading_11th_avg,'12th':reading_12th_avg})
reading_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [32]:
max_spending = max(school_summary['Per Student Budget']) #655dollars
min_spending = min(school_summary['Per Student Budget']) #578dollars

school_summary_reduced = school_summary.drop(columns=['School Type','Total Students','Total School Budget'])

bin1 = school_summary_reduced.loc[school_summary_reduced['Per Student Budget'] < 585]

bin2 = school_summary_reduced.loc[(school_summary_reduced['Per Student Budget'] >= 585) 
                                  & (school_summary_reduced['Per Student Budget'] < 610)]

bin3 = school_summary_reduced.loc[(school_summary_reduced['Per Student Budget'] >= 610) 
                                  & (school_summary_reduced['Per Student Budget'] < 635)]

bin4 = school_summary_reduced.loc[(school_summary_reduced['Per Student Budget'] >= 635) 
                                  & (school_summary_reduced['Per Student Budget'] < 660)]

row1 = bin1.mean().drop('Per Student Budget')
row2 = bin2.mean().drop('Per Student Budget')
row3 = bin3.mean().drop('Per Student Budget')
row4 = bin4.mean().drop('Per Student Budget')

columns = ['Spending Ranges (Per Student)','Average Math Score', 'Average Reading Score', '% Passing Math', 
           '% Passing Reading', '% Overall Passing']
scores_by_spending = pd.DataFrame(columns = columns)
scores_by_spending['Spending Ranges (Per Student)'] = ['<$585','$585-610','$610-635','$635-660']
scores_by_spending = scores_by_spending.set_index('Spending Ranges (Per Student)')
scores_by_spending.loc['<$585']=row1
scores_by_spending.loc['$585-610']=row2
scores_by_spending.loc['$610-635']=row3
scores_by_spending.loc['$635-660']=row4
scores_by_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.4554,83.9338,93.4601,96.6109,90.3695
$585-610,83.5997,83.8852,94.2309,95.9003,90.2163
$610-635,80.2,82.4254,80.0362,89.5361,72.6209
$635-660,77.8667,81.3688,70.3473,82.9956,58.8587


In [37]:
school_summary_reduced0 = school_summary.drop(columns=['School Type','Per Student Budget','Total School Budget'])

bin01 = school_summary_reduced0.loc[school_summary_reduced0['Total Students'] < 1000]

bin02 = school_summary_reduced0.loc[(school_summary_reduced0['Total Students'] >= 1000) 
                                  & (school_summary_reduced0['Total Students'] < 2000)]

bin03 = school_summary_reduced0.loc[(school_summary_reduced0['Total Students'] >= 2000) 
                                  & (school_summary_reduced0['Total Students'] < 5000)]


row01 = bin01.mean().drop('Total Students')
row02 = bin02.mean().drop('Total Students')
row03 = bin03.mean().drop('Total Students')

columns0 = ['School Size','Average Math Score', 'Average Reading Score', '% Passing Math', 
           '% Passing Reading', '% Overall Passing']
scores_by_size = pd.DataFrame(columns = columns0)
scores_by_size['School Size'] = ['Small(<1000)','Medium(1000-2000)','Large(2000-5000)']
scores_by_size = scores_by_size.set_index('School Size')
scores_by_size.loc['Small(<1000)']=row01
scores_by_size.loc['Medium(1000-2000)']=row02
scores_by_size.loc['Large(2000-5000)']=row03

scores_by_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small(<1000),83.8216,83.9298,93.5502,96.0994,89.8839
Medium(1000-2000),83.3747,83.8644,93.5997,96.7907,90.6215
Large(2000-5000),77.7464,81.3445,69.9634,82.7666,58.286


In [45]:
school_summary_reduced00 = school_summary.drop(columns=['Total Students','Per Student Budget','Total School Budget'])

bin001 = school_summary_reduced00.loc[school_summary_reduced00['School Type'] == 'Charter']

bin002 = school_summary_reduced00.loc[school_summary_reduced00['School Type'] == 'District'] 

row001 = bin001.mean()
row002 = bin002.mean()

columns00 = ['School Type','Average Math Score', 'Average Reading Score', '% Passing Math', 
               '% Passing Reading', '% Overall Passing']
scores_by_type = pd.DataFrame(columns = columns00)
scores_by_type['School Type'] = ['Charter','District']
scores_by_type = scores_by_type.set_index('School Type')
scores_by_type.loc['Charter']=row001
scores_by_type.loc['District']=row002

scores_by_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.4739,83.8964,93.6208,96.5865,90.4322
District,76.9567,80.9666,66.5485,80.7991,53.6722


In [ ]:
#Report:
#1. Overall, more students succeed in reading than in math
#2. Spending per student is actually negatively correlated with overall passing %
#3. Small and medium schools have very similar overall passing rates, while students
#   at large schools are far less likely to pass